In [1]:
import numpy as np

# Definición de la clase Board, que representa el tablero del juego de 4 en raya.
class Board():
    def __init__(self):
              # Inicializa el estado del tablero como una matriz de 4x4 llena de ceros.
        self.state = np.zeros((4, 4))
 # Método para obtener todas las posiciones válidas donde se puede hacer un movimiento.
    def valid_moves(self):
        return [(i, j) for j in range(4) for i in range(4) if self.state[i, j] == 0]
# Método para actualizar el tablero con el movimiento de un jugador.
    def update(self, symbol, row, col):
              # Verifica si la posición está vacía.
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError("¡Movimiento ilegal!")
    # Método para comprobar si el juego ha terminado.
    def is_game_over(self):
        # Comprobar filas y columnas
        if (self.state.sum(axis=0) == 4).sum() >= 1 or (self.state.sum(axis=1) == 4).sum() >= 1:
            return 1
        if (self.state.sum(axis=0) == -4).sum() >= 1 or (self.state.sum(axis=1) == -4).sum() >= 1:
            return -1

        # Comprobar diagonales
        diag_sums = [
            sum([self.state[i, i] for i in range(4)]),
            sum([self.state[i, 4 - i - 1] for i in range(4)]),
        ]
        if diag_sums[0] == 4 or diag_sums[1] == 4:
            return 1
        if diag_sums[0] == -4 or diag_sums[1] == -4:
            return -1

        # Comprobar diagonales secundarias
        for offset in range(-3, 4):
            if np.trace(self.state, offset=offset) == 4:
                return 1
            if np.trace(self.state, offset=offset) == -4:
                return -1
            if np.trace(np.fliplr(self.state), offset=offset) == 4:
                return 1
            if np.trace(np.fliplr(self.state), offset=offset) == -4:
                return -1

        # Empate
        if len(self.valid_moves()) == 0:
            return 0

        # Seguir jugando
        return None

    # Método para reiniciar el tablero a su estado inicial.
    def reset(self):
        self.state = np.zeros((4, 4))


In [2]:
from tqdm import tqdm

# Definición de la clase Game, que maneja el flujo del juego.
class Game():
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board()

    # Método para realizar múltiples rondas de autoaprendizaje.
    def selfplay(self, rounds=100):
        wins = [0, 0]
         # Itera a través del número especificado de rondas, mostrando una barra de progreso.
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()
            for player in self.players:
                player.reset()
            game_over = False
            while not game_over:
                for player in self.players:
                    action = player.move(self.board)
                    self.board.update(player.symbol, action[0], action[1])
                    for player in self.players:
                        player.update(self.board)
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break
            self.reward()
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol:
                    wins[ix] += 1
        return wins
    # Método para asignar recompensas a los jugadores después de cada juego.
    def reward(self):
        winner = self.board.is_game_over()
        if winner == 0:  # empate
            for player in self.players:
                player.reward(0.5)
        else:  # le damos 1 recompensa al jugador que gana
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)


In [3]:
# Definición de la clase Agent, que representa a un agente que aprende a jugar 4 en raya.
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {}  # tabla con pares estado -> valor
        self.alpha = alpha        # learning rate
        self.positions = []       # guardamos todas las posiciones de la partida
        self.prob_exp = prob_exp  # probabilidad de explorar
 # Método para reiniciar las posiciones visitadas al comienzo de una nueva partida.
    def reset(self):
        self.positions = []
 # Método para decidir el próximo movimiento del agente.
    def move(self, board, explore=True):
        valid_moves = board.valid_moves()
        # exploración
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            # vamos a una posición aleatoria
            ix = np.random.choice(len(valid_moves))
            return valid_moves[ix]
        # explotación
        # vamos a la posición con más valor
        max_value = -1000
        best_row, best_col = None, None
        for row, col in valid_moves:
            next_board = board.state.copy()
            next_board[row, col] = self.symbol
            next_state = str(next_board.reshape(4*4))
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)
            if value >= max_value:
                max_value = value
                best_row, best_col = row, col
        return best_row, best_col
# Método para registrar el estado del tablero en la lista de posiciones.
    def update(self, board):
        self.positions.append(str(board.state.reshape(4*4)))
    # Método para actualizar la tabla de valores al final de la partida.
    def reward(self, reward):
        # al final de la partida (cuando recibimos la recompensa)
        # iteramos por todos los estados actualizando su valor en la tabla
        for p in reversed(self.positions):
            if self.value_function.get(p) is None:
                self.value_function[p] = 0
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            reward = self.value_function[p]


In [4]:
# Crea dos instancias de la clase Agent con sus parámetros de exploración
agent1 = Agent(prob_exp=0.5)  # Crea el agente 1 con una probabilidad de exploración de 0.5
agent2 = Agent()  # Crea el agente 2 con los valores predeterminados (prob_exp=0.5)

# Crea una instancia de la clase Game con los dos agentes creados
game = Game(agent1, agent2)

# Hace que los dos agentes jueguen 300,000 partidas de autoaprendizaje
game.selfplay(300000)

100%|██████████| 300000/300000 [1:38:59<00:00, 50.51it/s]


[102213, 84592]

In [5]:
import pandas as pd

# Ordenar la función de valor del agente1 en función del valor
funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)

# Crear un DataFrame con los estados y sus valores
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

# Mostrar la tabla
tabla


,estado,valor
0,[ 0. 0. 0. 1. 0. -1. -1. 1. 0. 0. -1. ...,1.0
1,[ 0. 0. 0. 1. 0. 0. -1. 1. 0. 0. 0. ...,1.0
2,[ 0. 0. 0. 1. 0. 0. -1. 1. 0. -1. 0. ...,1.0
3,[ 0. 0. 0. 1. 0. 0. -1. 1. 0. 0. -1. ...,1.0
4,[-1. 0. -1. 1. 0. 0. -1. 1. 0. 0. 0. ...,1.0
...,...,...
1086961,[-1. -1. 1. 1. -1. -1. 1. -1. -1. 1. 0. ...,0.0
1086962,[-1. -1. 1. 1. -1. 0. 1. -1. -1. 1. 0. ...,0.0
1086963,[-1. 1. 1. 1. 1. -1. 1. 1. 0. -1. -1. ...,0.0
1086964,[ 0. 1. 1. 1. 1. -1. 1. 1. 0. -1. -1. ...,0.0


In [6]:
import pickle  # Importa el módulo pickle, que se utiliza para la serialización y deserialización de objetos en Python.

# Guardar la función de valor del agent1 en un archivo
# Abre un archivo llamado 'agente.pickle' en modo escritura binaria ('wb').
with open('agente.pickle', 'wb') as handle:
    # Usa pickle.dump para serializar y guardar la función de valor del agent1 en el archivo.
    # pickle.HIGHEST_PROTOCOL se utiliza para asegurar la máxima eficiencia en la serialización.
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)